In [81]:
import curses
import time
import RPi.GPIO as GPIO
import os

PIN14 = 14
PIN15 = 15
PIN23 = 23
PIN24 = 24

LASER = 4
HZ = 1000

TOY = 25

def forward(pwm1, pwm2, speed):
    if pwm1 is not None and  pwm2 is not None:
        pwm1.stop()
        pwm2.stop()
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN15, GPIO.LOW)
    GPIO.output(PIN23, GPIO.LOW)
    GPIO.output(PIN24, GPIO.LOW)
    pwm1 = GPIO.PWM(PIN14, HZ)
    pwm2 = GPIO.PWM(PIN23, HZ)
    pwm1.start(speed)
    pwm2.start(speed)
    return pwm1, pwm2

def backward(pwm1, pwm2, speed):
    if pwm1 is not None and  pwm2 is not None:
        pwm1.stop()
        pwm2.stop()
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN15, GPIO.LOW)
    GPIO.output(PIN23, GPIO.LOW)
    GPIO.output(PIN24, GPIO.LOW)
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN23, GPIO.LOW)
    pwm1 = GPIO.PWM(PIN15, HZ)
    pwm2 = GPIO.PWM(PIN24, HZ)
    pwm1.start(speed)
    pwm2.start(speed)
    return pwm1, pwm2

def right(pwm1, pwm2, speed):
    if pwm1 is not None and  pwm2 is not None:
        pwm1.stop()
        pwm2.stop()
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN15, GPIO.LOW)
    GPIO.output(PIN23, GPIO.LOW)
    GPIO.output(PIN24, GPIO.LOW)
    pwm1 = GPIO.PWM(PIN14, HZ)
    pwm2 = GPIO.PWM(PIN24, HZ)
    pwm1.start(speed)
    pwm2.start(speed)
    return pwm1, pwm2

def left(pwm1, pwm2, speed):
    if pwm1 is not None and  pwm2 is not None:
        pwm1.stop()
        pwm2.stop()
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN14, GPIO.LOW)
    pwm1 = GPIO.PWM(PIN15, HZ)
    pwm2 = GPIO.PWM(PIN23, HZ)
    pwm1.start(speed)
    pwm2.start(speed)
    return pwm1, pwm2

def stop(pwm1, pwm2):
    if pwm1 is not None and  pwm2 is not None:
        pwm1.stop()
        pwm2.stop()
    GPIO.output(PIN14, GPIO.LOW)
    GPIO.output(PIN15, GPIO.LOW)
    GPIO.output(PIN23, GPIO.LOW)
    GPIO.output(PIN24, GPIO.LOW)
    pwm1 = None
    pwm2 = None
    return pwm1, pwm2

def update_laser(laser):
    if laser:
        GPIO.output(LASER, GPIO.HIGH)
    else:
        GPIO.output(LASER, GPIO.LOW)

def change_toy(toy):
    if toy:
        GPIO.output(TOY, GPIO.HIGH)
    else:
        GPIO.output(TOY, GPIO.LOW)


GPIO.cleanup()

# Declare the GPIO settings
GPIO.setmode(GPIO.BCM)

# set up GPIO pins
GPIO.setup(PIN14, GPIO.OUT)
GPIO.setup(PIN15, GPIO.OUT)
GPIO.setup(PIN23, GPIO.OUT)
GPIO.setup(PIN24, GPIO.OUT)
GPIO.setup(LASER, GPIO.OUT)
GPIO.setup(TOY, GPIO.OUT)

GPIO.output(PIN14, GPIO.LOW)
GPIO.output(PIN15, GPIO.LOW)

GPIO.output(PIN23, GPIO.LOW)
GPIO.output(PIN24, GPIO.LOW)
GPIO.output(4, GPIO.LOW)
 
#set GPIO Pins
GPIO_TRIGGER = 20
GPIO_ECHO = 21
 
#set GPIO direction (IN / OUT)
GPIO.setup(GPIO_TRIGGER, GPIO.OUT)
GPIO.setup(GPIO_ECHO, GPIO.IN)

In [4]:
def distance():
    # set Trigger to HIGH
    GPIO.output(GPIO_TRIGGER, GPIO.HIGH)
 
    # set Trigger after 0.01ms to LOW
    time.sleep(0.00001)
    GPIO.output(GPIO_TRIGGER, GPIO.LOW)
 
    StartTime = time.time()
    StopTime = time.time()
 
    # save StartTime
    while GPIO.input(GPIO_ECHO) == 0:
        StartTime = time.time()
 
    # save time of arrival
    while GPIO.input(GPIO_ECHO) == 1:
        StopTime = time.time()
 
    # time difference between start and arrival
    TimeElapsed = StopTime - StartTime
    # multiply with the sonic speed (34300 cm/s)
    # and divide by 2, because there and back
    distance = (TimeElapsed * 34300) / 2
 
    return distance

In [8]:
# speed = 13cm/s
def go_forward(x):
    speed = 30
    v = 14.3
    t = x/v
    pwm1, pwm2 = None, None
    pwm1, pwm2 = stop(pwm1, pwm2)
    pwm1, pwm2 = backward(pwm1, pwm2, speed)
    time.sleep(t)
    pwm1, pwm2 = stop(pwm1, pwm2)
def go_time(t):
    pwm1, pwm2 = None, None
    pwm1, pwm2 = stop(pwm1, pwm2)
    pwm1, pwm2 = backward(pwm1, pwm2, speed)
    time.sleep(t)
    pwm1, pwm2 = stop(pwm1, pwm2)

In [9]:
import os
import sys
import time
import smbus
import math

from imusensor.MPU9250 import MPU9250

In [10]:
address = 0x68
bus = smbus.SMBus(1)
imu = MPU9250.MPU9250(bus, address)
imu.begin()

The name is wrong [113]


1

In [60]:
imu.loadCalibDataFromFile('/home/pi/imu.json')

In [61]:
def robot_heading():
    imu.readSensor()
    xyz = imu.MagVals
    x = xyz[0]
    y = xyz[1]
    heading = (math.atan2(y, x)*180)/math.pi

    if heading < 0:
        heading+=360
    return heading

In [82]:
def turn_left(t):
    speed = 30
    pwm1, pwm2 = None, None
    pwm1, pwm2 = stop(pwm1, pwm2)
    pwm1, pwm2 = left(pwm1, pwm2, speed)
    time.sleep(t)
    pwm1, pwm2 = stop(pwm1, pwm2)

def turn_right(t):
    speed = 30
    pwm1, pwm2 = None, None
    pwm1, pwm2 = stop(pwm1, pwm2)
    pwm1, pwm2 = right(pwm1, pwm2, speed)
    time.sleep(t)
    pwm1, pwm2 = stop(pwm1, pwm2)

In [91]:
turn_left(10)